# Quantum Software
*Hilary Term 2023*

---
This notebook implements the standard and block Gaussian elimination methods for CNOT circuit synthesis in Python3.

Run the cell below to import the necessary libraries.

In [1]:
import numpy as np

# This implementation uses PyZX to represent quantum circuits.
from pyzx import Circuit, draw, compare_tensors

---
## Example Circuit

First, validate that the CNOT circuit given in the report for this work is indeed correct.

In [2]:
circuit_example = Circuit(6)

# Left-hand side of figure.
circuit_example.add_gate('CNOT', 2, 1)
circuit_example.add_gate('CNOT', 5, 4)
circuit_example.add_gate('CNOT', 1, 0)
circuit_example.add_gate('CNOT', 4, 1)
circuit_example.add_gate('CNOT', 5, 2)

# Right-hand side of figure.
circuit_example.add_gate('CNOT', 2, 3)
circuit_example.add_gate('CNOT', 3, 2)
circuit_example.add_gate('CNOT', 2, 4)
circuit_example.add_gate('CNOT', 1, 2)
circuit_example.add_gate('CNOT', 2, 1)
circuit_example.add_gate('CNOT', 0, 2)
circuit_example.add_gate('CNOT', 2, 0)
circuit_example.add_gate('CNOT', 0, 4)

draw(circuit_example)
print(f"Number of CNOTs: {circuit_example.stats_dict()['cnot']}")

Number of CNOTs: 13


The cell below defines a function that computes the parity matrix for a given CNOT circuit.

In [3]:
def parity_matrix(circuit: Circuit) -> np.ndarray:
     # Start with the identity matrix.
     n = circuit_example.stats_dict()['qubits']
     matrix = np.eye(n, dtype=int)

     # Add the rows corresponding to the control and target of each CNOT gate.
     for gate in circuit.gates:
          assert gate.name == 'CNOT'
          matrix[gate.target, :] ^=  matrix[gate.control, :]

     return matrix

As expected, the circuit implements the desired parity matrix.

In [4]:
matrix_example = parity_matrix(circuit_example)
print(matrix_example)

[[0 1 1 1 1 1]
 [0 0 0 1 0 0]
 [1 0 0 1 1 1]
 [0 0 1 1 0 1]
 [0 1 1 0 0 0]
 [0 0 0 0 0 1]]


---
## CNOT Synthesis Algorithms
Before implementing the CNOT circuit synthesis algorithms detailed in the report, first define a function that generates random invertible parity matrices.

In [5]:
def random_parity_matrix(n: int, depth: int) -> np.ndarray:
    # Start with the identity matrix.
    matrix = np.eye(n, dtype=np.int)

    # Iterate for the given number of CNOT gates to generate.
    for _ in range(depth):
        # Get two random rows.
        row1 = np.random.randint(0, n)
        row2 = np.random.randint(0, n)

        # Make sure that the rows are not the same.
        while row1 == row2:
            row1 = np.random.randint(0, n)
            row2 = np.random.randint(0, n)

        # Add row2 to row1 to simulate the action of a CNOT gate.
        # I.e., elementary matrix multiplication.
        matrix[row1, :] ^= matrix[row2, :]

    return matrix

Run the cell below to generate a random parity matrix.

In [6]:
n = 6
matrix_random = random_parity_matrix(n, depth=n**2)

The following cell defines a function that validates the output of a CNOT circuit synthesis algorithm.

In [7]:
def validate_syth(matrix_orig: np.ndarray, matrix_new: np.ndarray, n: int, circuit: Circuit) -> None:
    # Make sure the matrix was reduced to the identity matrix.
    assert np.all(matrix_new == np.eye(n))

    # Check that the parity matrix of the new circuit matches the input parity matrix.
    parity = parity_matrix(circuit)
    assert np.all(matrix_orig == parity)

### Standard Gaussian Elimination

The following is an implementation of the standard Gaussian elimination method for CNOT circuit synthesis.

In [8]:
def syth_gauss(matrix: np.ndarray, n: int) -> Circuit:
    # Create an empty circuit.
    matrix_orig, matrix_new = np.copy(matrix), np.copy(matrix)
    circuit = Circuit(n)

    # Eliminate ones from the lower half of the matrix.
    for col in range(n):
        diag = matrix_new[col, col]

        # Iterate over the rows below the diagonal.
        for row in range(col + 1, n):
            # If there is a one below the diagonal, it must be eliminated.
            if matrix_new[row, col] == 1:
                # Use the one to place a one on the diagonal, if necessary.
                if diag == 0:
                    matrix_new[col, :] ^= matrix_new[row, :]
                    circuit.prepend_gate('CNOT', row, col)
                    diag = 1

                # Eliminate the one below the diagonal.
                matrix_new[row, :] ^= matrix_new[col, :]
                circuit.prepend_gate('CNOT', col, row)

    # Eliminate ones from the upper half of the matrix.
    for col in range(n - 1, 0, -1):
        for row in range(col - 1, -1, -1):
            if matrix_new[row, col] == 1:
                matrix_new[row, :] ^= matrix_new[col, :]
                circuit.prepend_gate('CNOT', col, row)

    # Check that the matrix was correctly reduced and return the corresponding circuit.
    validate_syth(matrix_orig, matrix_new, n, circuit)
    return circuit

The cell below synthesises a CNOT circuit for the example parity matrix using the standard Gaussian elimination method.

In [9]:
circuit_example_gauss = syth_gauss(matrix_example, n)
draw(circuit_example_gauss)
print(f"Number of CNOTs: {circuit_example_gauss.stats_dict()['cnot']}")

Number of CNOTs: 14


The cell below synthesises a CNOT circuit for the random parity matrix using the standard Gaussian elimination method.

In [10]:
circuit_random_gauss = syth_gauss(matrix_random, n)
draw(circuit_random_gauss)

print(f"Number of CNOTs: {circuit_random_gauss.stats_dict()['cnot']}")

Number of CNOTs: 20


### Block Gaussian Elimination

The following is an implementation of the block Gaussian elimination method (Patel-Markov-Hayes algorithm) for CNOT circuit synthesis.

In [11]:
def syth_pmh(matrix: np.ndarray, n: int, m: int) -> Circuit:
    matrix_orig, matrix_new = np.copy(matrix), np.copy(matrix)

    # Get the gates that implement the lower and upper triangles of the matrix.
    lower_gates = _syth_pmh_lower(matrix_new, n, m)
    upper_gates = _syth_pmh_lower(matrix_new.T, n, m)

    # First apply the gates from the upper triangle.
    # Note that the control and target are swapped.
    circuit = Circuit(n)
    for control, target in upper_gates:
        circuit.add_gate('CNOT', target, control)

    # Then apply the gates from the lower triangle in reverse order.
    for control, target in lower_gates[::-1]:
        circuit.add_gate('CNOT', control, target)

    # Check that the matrix was correctly reduced and return the corresponding circuit.
    validate_syth(matrix_orig, matrix_new, n, circuit)
    return circuit


def _syth_pmh_lower(matrix, n, m):
    gates = []

    # Iterate over each section of size `m`.
    num_sections = int(np.ceil(n / m))
    for section in range(1, num_sections + 1):
        section_start = (section - 1) * m
        section_end = min(section_start + m, n)

        # Map sub-row patterns to the corresponding row in which they first appear.
        pattern_to_row = {}
        for row in range(n - 1, section_start - 1, -1):
            pattern = tuple(matrix[row, section_start: section_end])
            pattern_to_row[pattern] = row

        # Iterate over the rows below the first diagonal for the section.
        for row in range(section_start + 1, n):
            # Get the row corresponding to the first occurrence of this sub-row's pattern.
            pattern = tuple(matrix[row, section_start: section_end])
            match = pattern_to_row[pattern]

            # Check if this sub-row contains a duplicate pattern (cannot be all zeros).
            if 1 in pattern and match != row:
                matrix[row, :] ^= matrix[match, :]
                gates.append((match, row))

        # Iterate over the columns in the section.
        for col in range(section_start, section_end):
            diag = matrix[col, col]

            # Iterate over the rows below the diagonal for the column.
            for row in range(col + 1, n):
                # If there is a one below the diagonal, it must be eliminated.
                if matrix[row, col] == 1:
                    # Use the one to place a one on the diagonal if necessary.
                    if diag == 0:
                        matrix[col, :] ^= matrix[row, :]
                        gates.append((row, col))
                        diag = 1

                    # Eliminate the one below the diagonal.
                    matrix[row, :] ^= matrix[col, :]
                    gates.append((col, row))

    return gates

The cell below synthesises a CNOT circuit for the example parity matrix using the PMH algorithm. In this case, the PMH algorithm produces a circuit with one fewer CNOT gate than standard Gaussian elimination.

In [12]:
circuit_example_pmh = syth_pmh(matrix_example, n, m=2)
draw(circuit_example_pmh)

# Make sure the outputs of the Gaussian elimination and PHM algorithms match.
assert compare_tensors(circuit_example_gauss, circuit_example_pmh, preserve_scalar=True)

print(f"Number of CNOTs: {circuit_example_pmh.stats_dict()['cnot']}")

Number of CNOTs: 13


The cell below synthesises a CNOT circuit for the random parity matrix using the PMH algorithm.

In [13]:
circuit_random_pmh = syth_pmh(matrix_random, n, m=2)
draw(circuit_random_pmh)

# Make sure the outputs of the Gaussian elimination and PHM algorithms match.
assert compare_tensors(circuit_random_gauss, circuit_random_pmh, preserve_scalar=True)

print(f"Number of CNOTs: {circuit_random_pmh.stats_dict()['cnot']}")

Number of CNOTs: 17
